# Lecture 5.3: Evaluation

This lecture, we are going to evaluate sklearn models.

**Learning goals:**
- split train, validation, and test set with sklearn
- run end to end machine learning experiments
- compare model quality
- tune a hyperparameter

##  1. Introduction

An estimated [\$70 million](https://en.wikipedia.org/wiki/Counterfeit_United_States_currency) in counterfeit bills are currently in circulation in the USA. That's quite a hustle 😎💰. The Federal Reserve doesn't like it however, and wants our help detecting fake banknotes. This can be a hard task: tiny defects are tough to spot, and counterfeiters constantly change their techniques. 

Machine Learning models can help, because they performs particularly well on unseen data. The [banknote authentication dataset](https://archive.ics.uci.edu/ml/datasets/banknote+authentication) frames this challenge as a binary classification task.  Let's evaluate and compare ML models trained on this fake/genuine banknote dataset.

Let's follow the checklist from the lecture slides.

## 2. 🤔 define ML task

As defined above: we are trying to solve a _binary classification task_: fake vs genuine banknotes.

## 3. 🔍 assess model feasibility

Detecting fake banknote is a pretty hard problem, but can be done by human experts. ML is also particularly good at detecting low level patterns in images. We also know that this is a solved problem, and have a dataset available. This task is therefore feasible!

## 4. 📂 find data you want to do well on

The banknote authentication dataset is a good representation of the bills we might encounter in the "wild". We can load it into a `Dataframe`:


In [ ]:
import pandas as pd

df = pd.read_csv('banknote.csv')
df.head()

In [ ]:
df.describe()

Each row is a banknote example, and there are four numerical features, `feature_i`, and one binary label, `is_fake`. It might be surprising that these examples aren't images. Instead, they are [wavelet transforms](https://en.wikipedia.org/wiki/Wavelet_transform) of the banknote pictures.

All four features have $mean = 0$ and $std = 1$ : they have already been standardized. The `count` row of the summary statistic table shows that there are no missing values. This means no further data preprocessing is necessary, and we can directly train our classifiers. 🏃‍♂️

## 6. ✂️ split a test set and set it aside

We usually jump straight into converting this `DataFrame` to features, which we then use to `.fit()` our model. This time however, we first split a test set.

sklearn makes this easy with the `train_test_split` function. The [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) mentions that it can split many different inputs:

> Allowed inputs are lists, numpy arrays, scipy-sparse matrices or pandas dataframes.

We choose to split our `DataFrame` 80%/20%:

In [ ]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.20, random_state=777)
print(f'total size: {len(df)}, train set size: {len(train_df)}, test set size: {len(test_df)}')

We choose to "set aside" the test set for later use. This prevents us from accidentally using data from the test set during development:

In [ ]:
train_df.to_csv('banknote_train.csv', index=False)
test_df.to_csv('banknote_test.csv', index=False)

## 7. ✂️ split train & validation sets

We choose to split the validation set _lazily_ , meaning we won't save it to disk like the test set. This is fine, because validation sets _can_ be reused.  
i.e Our results won't be statistically compromised, if the split isn't the same for each round of experiments.

In [ ]:
df = pd.read_csv('banknote_train.csv')
train_df, val_df = train_test_split(df, test_size=0.20, random_state=4242)
print(f'train set size: {len(train_df)}, validation set size: {len(val_df)}')

## 8. 🎯 define single number metric

We are dealing with a balanced binary classification task, and therefore choose accuracy as our single number metric. This sole number will define our model quality.

## 9. 🔁 train + validate until happy with losses and metric(s)

We are now ready to experiment! Let's first create features and labels. We could use all four features, but it turns out that classification task is then too easy, and it wouldn't be interesting to compare training and validation metrics 😑.

So instead, we'll pick features 2 & 4 to spice up the task difficulty 🌶️

In [ ]:
def to_features(df):
    X = df[['feature_2', 'feature_4']].values
    y = df['is_fake'].values
    return X, y

X_train, y_train = to_features(train_df)
X_val, y_val = to_features(val_df)

For our first round of experiments, we'd like to know which type of model best solves our task. We'll use three different classifiers:
- logistic regression
- random forest
- SVM with RBF kernel

ℹ️ Don't worry if you haven't heard of the last two models before! There is a whole cornucopia of ML models out there - with new ones published everyday. But a good place to start is with all the [sklearn models](https://scikit-learn.org/stable/supervised_learning.html). Not only is their documentation a great place to learn about how they work and how to use them, but sklearn also has many additional resources, like this [classifier comparison chart](https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html). And they mostly follow the sklearn `.fit()` and `.predict()` API, so you can try them out straight away! The best way to learn about models is to explore which are commonly used when you encounter a new ML task. 

We fit these models on the training data:

In [ ]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

rf_clf = RandomForestClassifier(random_state=0).fit(X_train, y_train)
svm_clf = SVC(kernel='rbf', C=1000, random_state=0).fit(X_train, y_train)
lr_clf = LogisticRegression(random_state=0).fit(X_train, y_train)

We now want to calculate the _accuracy_ of our models. sklearn provides many metric functions in the [`sklearn.metrics`](https://scikit-learn.org/stable/modules/model_evaluation.html#classification-metrics) module, including `accuracy_score()`. It compares labels and predictions, so we can use the `.predict()` method of the model api. For example, for our linear regression model:

In [ ]:
from sklearn.metrics import accuracy_score

# predict labels
y_predict = lr_clf.predict(X_val)
# compare them to true labels
accuracy_score(y_val, y_predict)

69% accuracy, not bad!

🧠 What does `accuracy` represent? How does one calculate it?

Since it is such a common usecase, sklearn makes evaluation even easier by assigning default metrics to popular tasks and model types. For _classifiers_ , the default metric is already accuracy, so we can use the `.score()` method from the model api directly. sklearn will predict labels and compare them to the true labels for us:

In [ ]:
lr_clf.score(X_val, y_val)

Now that we know how to evaluate sklearn models, lets's compare all of our banknote classifiers:

In [ ]:
clfs = [rf_clf, svm_clf, lr_clf]

for clf in clfs:
    accuracy = clf.score(X_val, y_val)
    print(f'classifier: {type(clf).__name__}, validation accuracy: {accuracy}')

Wow, these models are pretty good! 🤩

Let's carry out a second round of experiments to determine optimal SVM hyperparameter. We're particularly interested in `C` which controls regularization.

💪 Train 6 SVMs, then compare their training & validation accuracy.
- use the `C` values listed below
- store the training accuracies in a list called `train_accuracies`
- store the validation accuracies in a list called `val_accuracies`
- use the unit test to debug and verify your code

In [ ]:
c_values = [0.1, 1, 10, 100, 1000]

# INSERT YOUR CODE HERE

In [ ]:
import math

def print_results(c_values, train_accuracies, val_accuracies):
    for c, train_acc, val_acc in zip(c_values, train_accuracies, val_accuracies):
        print(f'C: {c}, train acc: {train_acc}, val acc: {val_acc}')
        
        
def test_svm_C_tuning():
    assert train_accuracies, "Can't find train_accuracies. Did you use the correct variable name?"
    assert val_accuracies, "Can't find val_accuracies. Did you use the correct variable name?"
    assert len(train_accuracies) == 5, f"Expected 5 training accuracies, got {len(train_accuracies)}"
    assert len(val_accuracies) == 5, f"Expected 5 validation accuracies, got {len(val_accuracies)}"
    print_results(c_values, train_accuracies, val_accuracies)
    assert math.isclose(4.221208, sum(train_accuracies), rel_tol=1e-5), "Something is wrong with your training accuracy values"
    assert math.isclose(4.431818, sum(val_accuracies), rel_tol=1e-5), "Something is wrong with your validation accuracy values"
    print('Success! 🎉')
    
test_svm_C_tuning()

🧠 What is the best value for the hyperparameter `C`?

🧠 For which value of `C` does the SVM seem to start overfitting?

## 10. 📏 evaluate model on test set to get final metric

The SVM is our fake banknote detection model of choice. The International Monetary Fund would like guarantees about how well this model is going to perform in production. To know the expectation value of accuracy on unseen examples, we decide to use our _test set_ to measure the metric.

In [ ]:
test_df = pd.read_csv('banknote_test.csv')
X_test, y_test = to_features(test_df)
svm_clf.score(X_test, y_test)

🧠🧠 The test accuracy is slightly lower than the validation accuracy.
- What does test accuracy < validation accuracy usually indicate?
- Is the difference significant in this case? 
- How would you verify this?


## 4. Summary

Today, we learned about **evaluation methods**. First, we noted that training loss makes for a bad model quality metric, since it cannot detect **overfitting**. We introduced the idea of a held-out **test set** to better estimate generalization properties on unseen examples. We highlighted how test sets work if they are of the **same distribution** as the data we will encounter at prediction time, and if they are **large enough**. We then described how an independent test set can still be prone to overfitting if used as part of **model development**. Since machine learning development is **experimental** & **iterative** in nature, the data scientist introduces an **information leak** between the test set and the model hyperparameters. We introduced the **validation set** as a solution. We split the responsibilities of **comparing** models, and **assessing** models, which allows engineers to both develop and measure the quality machine learning solutions. We then showed that losses weren't always interpretable values, and introducted new **metrics**, like classification accuracy or regression MSE. We underlined the importance of choosing a **single number metric** to define model quality, and speed up model development. We then synthesized all these new workflows into a **ML development checklist**, which captures the steps of typical ML engineering experiments. Finally, we applied this checklist and built a viable ML solution from scratch for banknote authentication in sklearn.


# Resources

## Core Resources

- [Machine learning yearning](https://www.deeplearning.ai/machine-learning-yearning/)  
The Andrew Ng reference for ML engineering, including terse and practical sections about validation and test sets
- [sklearn on evaluation](https://scikit-learn.org/stable/modules/model_evaluation.html)  
Verbose official documentation on sklearn evaluation methods and apis 

## Additional Resources

- [Google ML crash course - accuracy](https://developers.google.com/machine-learning/crash-course/classification/accuracy)  
Intuitive explanation of the accuracy metric and its equation